How to import these things?


In [1]:
# imports
from requests import get
from bs4 import BeautifulSoup
import os


In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle
import wrangle as w
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import nltk
from collections import Counter
import random
import joblib
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import sys
import unicodedata
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/click/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# need to pull list of top 100 top star usernames
# example: 'https://github.com/underdash/underdash/blob/master/README.md'
# so far from looking, its a 'span class=username/projectname'
# To break it down
# url = 'https://github.com/' + 'username/projectname' + '/blob/master/README.md'


In [4]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = []
 

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_contents = requests.get(get_readme_download_url(contents)).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data2.json", "w"), indent=1)

In [5]:
url = 'https://github.com/search?q=stars%3A%3E0&s=stars&type=Repositories'

In [6]:
response_data = github_api_request(url)

In [7]:
response_data

{'payload': {'header_redesign_enabled': False,
  'results': [{'id': '28457823',
    'archived': False,
    'color': '#3178c6',
    'followers': 376284,
    'has_funding_file': True,
    'hl_name': 'freeCodeCamp/freeCodeCamp',
    'hl_trunc_description': "freeCodeCamp.org's open-source codebase and curriculum. Learn to code for free.",
    'language': 'TypeScript',
    'mirror': False,
    'owned_by_organization': True,
    'public': True,
    'repo': {'repository': {'id': 28457823,
      'name': 'freeCodeCamp',
      'owner_id': 9892522,
      'owner_login': 'freeCodeCamp',
      'updated_at': '2023-10-23T14:14:34.261Z',
      'has_issues': True}},
    'sponsorable': True,
    'topics': ['react',
     'nodejs',
     'javascript',
     'd3',
     'teachers',
     'community',
     'education',
     'programming',
     'curriculum',
     'math',
     'freecodecamp',
     'learn-to-code',
     'nonprofits',
     'careers',
     'certification',
     'hacktoberfest'],
    'type': 'Public',

In [8]:
processed_repos = set()

In [9]:
repo_links = []
for result in response_data["payload"]["results"]:
    owner_login = result["repo"]["repository"]["owner_login"]
    repo_name = result["repo"]["repository"]["name"]
    repo_url = f"/{owner_login}/{repo_name}"
    if repo_url not in processed_repos:
        repo_links.append(repo_url)
        processed_repos.add(repo_url)

In [10]:
repo_links

['/freeCodeCamp/freeCodeCamp',
 '/EbookFoundation/free-programming-books',
 '/sindresorhus/awesome',
 '/jwasham/coding-interview-university',
 '/996icu/996.ICU',
 '/public-apis/public-apis',
 '/kamranahmedse/developer-roadmap',
 '/donnemartin/system-design-primer',
 '/codecrafters-io/build-your-own-x',
 '/facebook/react']

In [11]:
processed_repos

{'/996icu/996.ICU',
 '/EbookFoundation/free-programming-books',
 '/codecrafters-io/build-your-own-x',
 '/donnemartin/system-design-primer',
 '/facebook/react',
 '/freeCodeCamp/freeCodeCamp',
 '/jwasham/coding-interview-university',
 '/kamranahmedse/developer-roadmap',
 '/public-apis/public-apis',
 '/sindresorhus/awesome'}

Code from Zac for acquire

In [12]:
def fetch_github_data(url):
    """
    Fetch data from a GitHub API endpoint.

    Parameters:
    - url (str): The URL of the API endpoint.

    Returns:
    - dict: The JSON response from the API.
    """

    # Define the base URL for the GitHub search API
    BASE_URL = "https://github.com/search?q=stars%3A%3E0+language%3A{language}&type=repositories&l={language}&p={page}"

    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:
        print(
            f"Status code: {response.status_code} - Rate limit exceeded. Ending script."
        )
        sys.exit()
    else:
        print(f"Status code: {response.status_code} - Failed to fetch data from {url}.")
        return None


In [13]:
BASE_URL = "https://github.com/search?q=stars%3A%3E0+language%3A{language}&type=repositories&l={language}&p={page}"

In [14]:


def extract_repo_links(data, processed_repos):
    """
    Extract repository links from the JSON response of a GitHub API endpoint.

    Parameters:
    - data (dict): The JSON response from the API.
    - processed_repos (set): A set of repository URLs that have already been processed.

    Returns:
    - list: A list of repository URLs that have not been processed before.
    """
    repo_links = []
    for result in data["payload"]["results"]:
        owner_login = result["repo"]["repository"]["owner_login"]
        repo_name = result["repo"]["repository"]["name"]
        repo_url = f"/{owner_login}/{repo_name}"
        if repo_url not in processed_repos:
            repo_links.append(repo_url)
            processed_repos.add(repo_url)
    return repo_links



In [15]:

def get_readme_content(repo):
    """
    Get the content of the README file for a repository.

    Parameters:
    - repo (str): The URL of the repository.

    Returns:
    - str: The content of the README file, or None if the file could not be found.
    """
    readme_url = "https://github.com" + repo + "/blob/master/README.md"
    response = requests.get(readme_url)
    soup = BeautifulSoup(response.content, "html.parser")
    readme_content = soup.select_one("article")
    return readme_content.get_text() if readme_content else None


In [16]:


def fetch_readmes(
    languages=["python"], num_repos=5, start_page=1, sleep_time=1, verbose=True
):
    """
    Fetch README files from GitHub repositories.

    Parameters:
    - languages (list): A list of programming languages to search for.
    - num_repos (int): The number of repositories to fetch README files from.
    - start_page (int): The page number to start searching from.
    - sleep_time (int): The number of seconds to wait between requests.
    - verbose (bool): Whether to print progress messages.

    Returns:
    - pandas.DataFrame: A DataFrame containing the language, repository URL, and README content for each fetched README file.
    """
    all_readmes = []
    processed_repos = set()

    for language in languages:
        if verbose:
            print(f"Processing {language} repositories...")
        page = start_page
        total_processed = 0

        while total_processed < num_repos:
            url = BASE_URL.format(language=language, page=page)
            data = fetch_github_data(url)
            if not data:
                print(f"Failed to fetch data for {language} on page {page}.")
                break

            repo_links = extract_repo_links(data, processed_repos)
            for repo in repo_links:
                if total_processed >= num_repos:
                    break
                readme_content = get_readme_content(repo)
                if readme_content:
                    all_readmes.append((language, repo, readme_content))
                    total_processed += 1
                    if verbose:
                        print(
                            f"Fetched README {total_processed} of {num_repos} for {language}."
                        )
                time.sleep(sleep_time)  # Add a delay between requests
            page += 1

        if num_repos > 10 and page <= num_repos // 10:
            if verbose:
                print(f"Fetching additional pages for {language}...")
            for i in range(page, num_repos // 10 + 1):
                url = BASE_URL.format(language=language, page=i)
                data = fetch_github_data(url)
                if not data:
                    print(f"Failed to fetch data for {language} on page {i}.")
                    break

                repo_links = extract_repo_links(data, processed_repos)
                for repo in repo_links:
                    if total_processed >= num_repos:
                        break
                    readme_content = get_readme_content(repo)
                    if readme_content:
                        all_readmes.append((language, repo, readme_content))
                        total_processed += 1
                        if verbose:
                            print(
                                f"Fetched README {total_processed} of {num_repos} for {language}."
                            )
                    time.sleep(sleep_time)  # Add a delay between requests

            if verbose:
                print(
                    f"Finished processing {language} repositories. Fetched a total of {total_processed} READMEs."
                )

    df = pd.DataFrame(all_readmes, columns=["language", "repo", "readme"])
    if verbose:
        print(f"Total README Count: {len(df)}")
    return df

In [17]:
fetch_github_data(url)

{'payload': {'header_redesign_enabled': False,
  'results': [{'id': '28457823',
    'archived': False,
    'color': '#3178c6',
    'followers': 376284,
    'has_funding_file': True,
    'hl_name': 'freeCodeCamp/freeCodeCamp',
    'hl_trunc_description': "freeCodeCamp.org's open-source codebase and curriculum. Learn to code for free.",
    'language': 'TypeScript',
    'mirror': False,
    'owned_by_organization': True,
    'public': True,
    'repo': {'repository': {'id': 28457823,
      'name': 'freeCodeCamp',
      'owner_id': 9892522,
      'owner_login': 'freeCodeCamp',
      'updated_at': '2023-10-23T14:14:34.261Z',
      'has_issues': True}},
    'sponsorable': True,
    'topics': ['react',
     'nodejs',
     'javascript',
     'd3',
     'teachers',
     'community',
     'education',
     'programming',
     'curriculum',
     'math',
     'freecodecamp',
     'learn-to-code',
     'nonprofits',
     'careers',
     'certification',
     'hacktoberfest'],
    'type': 'Public',

In [18]:
# python = fetch_readmes(languages=['python'], num_repos=100, verbose=True, sleep_time=5)

Processing python repositories...
Fetched README 1 of 100 for python.
Fetched README 2 of 100 for python.
Fetched README 3 of 100 for python.
Fetched README 4 of 100 for python.


KeyboardInterrupt: 

In [ ]:
# python.to_csv('python_readmes.csv')

In [19]:
python_df = pd.read_csv('python_readmes.csv')

In [20]:
python_df.head()

,Unnamed: 0,language,repo,readme
0,0,python,/ethereon/caffe-tensorflow,Caffe to TensorFlow\nConvert Caffe models to T...
1,1,python,/nanohop/sketch-to-react-native,Convert Sketch files to React Native component...
2,2,python,/jeffknupp/sandman,sandman\n\n\n\n\n\nHomepage\nVisit the home of...
3,3,python,/CreatCodeBuild/TensorFlow-and-DeepLearning-Tu...,TensorFlow and DeepLearning Tutorial\nThis is ...
4,4,python,/wrobstory/vincent,Status\n2016-06-18 Update\nIf you are interest...


In [21]:
# html = fetch_readmes(languages=['html'], num_repos=100, verbose=True, sleep_time=5)
# html.to_csv('html_readmes.csv')
html_df = pd.read_csv('html_readmes.csv')

In [22]:
html_df.head()

,Unnamed: 0,language,repo,readme
0,0,html,/tc39/proposal-optional-chaining,Optional Chaining for JavaScript\nStatus\nECMA...
1,1,html,/hackers-painters/samurai-native,\nBring web standards to native platform\nsamu...
2,2,html,/mritd/dockerfile,Dockerfile\n⚠️⚠️⚠️ Note: Some build work of do...
3,3,html,/superherojs/superherojs,Superhero.js\nMaintaining a large JavaScript c...
4,4,html,/quantifiedcode/python-anti-patterns,The Little Book of Python Anti-Patterns\nThis ...


In [98]:
# Check current rate limit with GitHub (does not count towards limit)
response = requests.get('https://api.github.com/rate_limit')
response.json()

{'resources': {'core': {'limit': 60,
   'remaining': 59,
   'reset': 1698076818,
   'used': 1,
   'resource': 'core'},
  'graphql': {'limit': 0,
   'remaining': 0,
   'reset': 1698077099,
   'used': 0,
   'resource': 'graphql'},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1698077099,
   'used': 0,
   'resource': 'integration_manifest'},
  'search': {'limit': 10,
   'remaining': 10,
   'reset': 1698073559,
   'used': 0,
   'resource': 'search'}},
 'rate': {'limit': 60,
  'remaining': 59,
  'reset': 1698076818,
  'used': 1,
  'resource': 'core'}}

In [97]:
# go = fetch_readmes(languages=['go'], num_repos=100, verbose=True, sleep_time=7)
# go.to_csv('go_readmes.csv')
# go_df = pd.read_csv('go_readmes.csv')

Processing go repositories...
Fetched README 1 of 100 for go.
Fetched README 2 of 100 for go.
Fetched README 3 of 100 for go.
Fetched README 4 of 100 for go.
Fetched README 5 of 100 for go.
Fetched README 6 of 100 for go.
Fetched README 7 of 100 for go.
Fetched README 8 of 100 for go.
Fetched README 9 of 100 for go.
Fetched README 10 of 100 for go.
Fetched README 11 of 100 for go.
Fetched README 12 of 100 for go.
Fetched README 13 of 100 for go.
Fetched README 14 of 100 for go.
Fetched README 15 of 100 for go.
Fetched README 16 of 100 for go.
Fetched README 17 of 100 for go.
Fetched README 18 of 100 for go.
Fetched README 19 of 100 for go.
Fetched README 20 of 100 for go.
Fetched README 21 of 100 for go.
Fetched README 22 of 100 for go.
Fetched README 23 of 100 for go.
Fetched README 24 of 100 for go.
Fetched README 25 of 100 for go.
Fetched README 26 of 100 for go.
Fetched README 27 of 100 for go.
Fetched README 28 of 100 for go.
Fetched README 29 of 100 for go.
Fetched README 30 of 1

SystemExit: 

In [23]:
# javascript = fetch_readmes(languages=['javascript'], num_repos=100, verbose=True, sleep_time=5)
# javascript.to_csv('javascript_readmes.csv')
javascript_df = pd.read_csv('javascript_readmes.csv')

In [24]:
# java = fetch_readmes(languages=['java'], num_repos=100, verbose=True, sleep_time=5)
# java.to_csv('java_readmes.csv')
java_df = pd.read_csv('java_readmes.csv')

In [25]:
df = pd.concat([java_df, python_df, javascript_df, html_df])

In [26]:
df.shape

(400, 4)

In [ ]:
# df.to_csv('all_readmes.csv')

In [ ]:
df.head()

,Unnamed: 0,language,repo,readme
0,0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [125]:
df = pd.read_csv('all_readmes.csv')

In [126]:
df.head()

,Unnamed: 0.1,Unnamed: 0,language,repo,readme
0,0,0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,1,1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,2,2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,3,3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,4,4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [127]:
df.columns = df.columns.str.lower()

In [128]:
df.columns

Index(['unnamed: 0.1', 'unnamed: 0', 'language', 'repo', 'readme'], dtype='object')

In [129]:
df.columns = df.columns.str.replace(' ', '').str.replace(':', '')

In [130]:
df.drop(columns =['unnamed0.1', 'unnamed0'], inplace=True)

In [131]:
df.head()

,language,repo,readme
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


## Prepare

In [132]:
df.head()

,language,repo,readme
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [133]:
def basic_clean(text):

    text = unicodedata.normalize('NFKD', text.lower().replace('\\n', ' '))\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
    return re.sub(r"[^a-z0-9'\s]", '', text)

def stem(text):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in text.split()]
    return ' '.join(stems)

def lemmatize(text):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    return ' '.join(lemmas)

def remove_stopwords(text):
    additional_stopwords = ['java', 'javascript', 'html', 'python']
    tokenizer = ToktokTokenizer()
    stopword_list = stopwords.words('english')
    stopword_list = set(stopword_list).union(set(additional_stopwords))
    stopword_list.remove('no')
    stopword_list.remove('not')
    
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [t for t in tokens if t not in stopword_list]
    return ' '.join(filtered_tokens)

In [134]:
df.head()

,language,repo,readme
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp..."
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...


In [135]:
df['readme_clean'] = df.readme.apply(basic_clean).apply(remove_stopwords)

In [136]:
df.head()

,language,repo,readme,readme_clean
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice no longer supported near...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice no longer maintained n...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...


In [137]:
df['readme_stemmed'] = df.readme_clean.apply(stem).apply(remove_stopwords)

In [138]:
df.head()

,language,repo,readme,readme_clean,readme_stemmed
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice no longer supported near...,august 2016 roboguic no longer support nearli ...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...,android stackblur android stackblur librari pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...,butterknifezelezni simpl plugin android studio...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice no longer maintained n...,dragsortlistview notic no longer maintain not ...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...,nativ navig disclaim project current beta mani...


In [139]:
df['readme_lem'] = df.readme_clean.apply(lemmatize).apply(remove_stopwords)

In [140]:
df.head()

,language,repo,readme,readme_clean,readme_stemmed,readme_lem
0,java,/roboguice/roboguice,"As of August 2016, RoboGuice is no longer supp...",august 2016 roboguice no longer supported near...,august 2016 roboguic no longer support nearli ...,august 2016 roboguice no longer supported near...
1,java,/kikoso/android-stackblur,Android StackBlur\nAndroid StackBlur is a libr...,android stackblur android stackblur library pe...,android stackblur android stackblur librari pe...,android stackblur android stackblur library pe...
2,java,/avast/android-butterknife-zelezny,ButterKnifeZelezny\nSimple plug-in for Android...,butterknifezelezny simple plugin android studi...,butterknifezelezni simpl plugin android studio...,butterknifezelezny simple plugin android studi...
3,java,/bauerca/drag-sort-listview,DragSortListView\nNOTICE: No longer maintained...,dragsortlistview notice no longer maintained n...,dragsortlistview notic no longer maintain not ...,dragsortlistview notice no longer maintained n...
4,java,/airbnb/native-navigation,Native Navigation\n\n\n\n\nDISCLAIMER\nThis pr...,native navigation disclaimer project currently...,nativ navig disclaim project current beta mani...,native navigation disclaimer project currently...


## Explore

show counts and ratios for the dataframe

In [141]:
def show_counts_and_ratios(df, column):
    """
    Takes in a dataframe and a string of a single column
    Returns a dataframe with absolute value counts and percentage value counts
    """
    labels = pd.concat([df[column].value_counts(),
                    df[column].value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    labels
    return labels




In [142]:
show_counts_and_ratios(df, 'language')

,n,percent
language,,
java,100,0.25
python,100,0.25
javascript,100,0.25
html,100,0.25


In [143]:
java_words = basic_clean(lemmatize(remove_stopwords(' '.join(df[df.language == 'java'].readme))))
html_words = basic_clean(lemmatize(remove_stopwords(' '.join(df[df.language == 'html'].readme))))
javascript_words = basic_clean(lemmatize(remove_stopwords(' '.join(df[df.language == 'javascript'].readme))))
python_words = basic_clean(lemmatize(remove_stopwords(' '.join(df[df.language == 'python'].readme))))

In [144]:
java_words = java_words.split()
javascript_words = javascript_words.split()
python_words = python_words.split()
html_words = html_words.split()

In [147]:
java_freq = pd.Series(java_words).value_counts()
javascript_freq = pd.Series(javascript_words).value_counts()
python_freq = pd.Series(python_words).value_counts()
html_freq = pd.Series(html_words).value_counts()


In [151]:
all_words = basic_clean(lemmatize(remove_stopwords(' '.join(df.readme))))

In [152]:
all_words_freq = pd.Series(all_words).value_counts()

In [148]:
java_freq.head()

android    504
'          495
license    330
the        264
new        261
Name: count, dtype: int64

In [149]:
javascript_freq.head()

'           6677
124         1172
function     761
const        646
the          627
Name: count, dtype: int64

In [150]:
html_freq.head(10)

'          1383
the         272
notice      202
use         202
install     166
not         152
file        148
this        143
using       132
i           126
Name: count, dtype: int64

In [153]:
word_counts = (pd.concat([html_freq, python_freq, java_freq, javascript_freq, all_words_freq], axis=1, sort=True).set_axis(['html', 'python', 'java', 'javascript', 'all'], axis=1).fillna(0).apply(lambda s: s.astype(int)))

In [154]:
word_counts.head()

,html,python,java,javascript,all
',1383,3430,495,6677,0
0,86,201,53,261,0
00,0,16,0,12,0
000,1,7,0,11,0
0000,0,4,0,1,0
